In [1]:
!git clone https://github.com/pattan99/Handwritten-Mathematical-Expression-Recognition.git
%cd Handwritten-Mathematical-Expression-Recognition

Cloning into 'Handwritten-Mathematical-Expression-Recognition'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 64 (delta 9), reused 55 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (64/64), 10.68 MiB | 15.85 MiB/s, done.
Resolving deltas: 100% (9/9), done.
/content/Handwritten-Mathematical-Expression-Recognition


In [2]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.0 MB/s eta 0:00:00


In [3]:
%%capture
!mkdir data/mathwriting-2024

# !gdown "https://drive.google.com/uc?id=1M8nyh6Lg5V1z7m4Mn2Lo5BiD0EbIcAAV"
# !gdown "https://drive.google.com/uc?id=1wZEcyfDHSNVagh4dOJCtvCo_3JQ5pwEX"
!gdown "https://drive.google.com/uc?id=1VYZErLFOZ-ZbfDhsGm40OV2O9Fui6T_a"

# !mv train.zip data/mathwriting-2024
# !mv val.zip data/mathwriting-2024
!mv test.zip data/mathwriting-2024

# !unzip data/mathwriting-2024/train.zip -d data/mathwriting-2024
# !unzip data/mathwriting-2024/val.zip -d data/mathwriting-2024
!unzip data/mathwriting-2024/test.zip -d data/mathwriting-2024

In [4]:
%%capture
# !gdown --id 1ROWlHZpVfcIVEh8NQkZHl2LXPuvL7VRK
# !gdown --id 1Y5G3xnODYhnl401sqxRaElpcOYtSfFiX
!gdown --id 1E2UHNkvKcZCQ2DHUFGDmatmzYfIUuOuJ

# !mv train_img.zip data/
# !mv valid_img.zip data/
!mv test_img.zip data/

# !unzip data/train_img.zip -d data/
# !unzip data/valid_img.zip -d data/
!unzip data/test_img.zip -d data/

In [5]:
import os
MATHWRITING_ROOT_DIR='data/mathwriting-2024'
TRAIN_DIR = os.path.join(MATHWRITING_ROOT_DIR, 'train')
VAL_DIR = os.path.join(MATHWRITING_ROOT_DIR, 'valid')
TEST_DIR = os.path.join(MATHWRITING_ROOT_DIR, 'test')
SYMBOL_DIR = os.path.join(MATHWRITING_ROOT_DIR, 'symbols')

In [6]:
import pandas as pd
train_df = pd.read_csv("data/train.csv")
train_df['file_path'] = train_df['file_path'].apply(lambda x: MATHWRITING_ROOT_DIR+'/'+x.split('/')[-2]+'/'+x.split('/')[-1].replace('.inkml','.bin'))
train_df['image_path'] = train_df['file_path'].apply(lambda x: x.replace('train', 'train_img').replace('.bin','.png'))
# train_df['file_path'] = train_df['file_path'].apply(lambda x: x.replace('train', 'train_merged'))

valid_df = pd.read_csv("data/val.csv")
valid_df['file_path'] = valid_df['file_path'].apply(lambda x: MATHWRITING_ROOT_DIR+'/'+x.split('/')[-2]+'/'+x.split('/')[-1].replace('.inkml','.bin'))
valid_df['image_path'] = valid_df['file_path'].apply(lambda x: x.replace('valid', 'valid_img').replace('.bin','.png'))
valid_df['file_path'] = valid_df['file_path'].apply(lambda x: x.replace('valid', 'val'))

test_df = pd.read_csv("data/test.csv")
test_df['file_path'] = test_df['file_path'].apply(lambda x: MATHWRITING_ROOT_DIR+'/'+x.split('/')[-2]+'/'+x.split('/')[-1].replace('.inkml','.bin'))
test_df['image_path'] = test_df['file_path'].apply(lambda x: x.replace('test', 'test_img').replace('.bin','.png'))
# test_df['file_path'] = test_df['file_path'].apply(lambda x: x.replace('test', 'test_merged'))

In [7]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from torch.utils.data import DataLoader
from dataloader import StrokeDataset, collate_fn

In [16]:
train_dataset = StrokeDataset(data_dir=MATHWRITING_ROOT_DIR, labels_df=train_df)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2, collate_fn=collate_fn)
valid_dataset = StrokeDataset(data_dir=MATHWRITING_ROOT_DIR, labels_df=test_df)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=2, collate_fn=collate_fn)

Vocab size: 257
Vocab size: 257
Vocab size: 257


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [10]:
from model import init_clip_model, load_clip_model
from model import OnHWRTransformer
from trainer import train, evaluate

In [11]:
!gdown 1xNin029kMH5OmC64M0bEt7aSdJM6ioTq

Downloading...
From (original): https://drive.google.com/uc?id=1xNin029kMH5OmC64M0bEt7aSdJM6ioTq
From (redirected): https://drive.google.com/uc?id=1xNin029kMH5OmC64M0bEt7aSdJM6ioTq&confirm=t&uuid=e93ad8ec-3c32-43bc-b153-29998eb73cea
To: /content/Handwritten-Mathematical-Expression-Recognition/latest_line_1_patch_16_9.pt
100% 1.25G/1.25G [00:13<00:00, 92.0MB/s]


In [12]:
clip_model = init_clip_model(device)
clip_model = load_clip_model('latest_line_1_patch_16_9.pt', device)

Vocab size: 257
Vocab size: 257


/content/Handwritten-Mathematical-Expression-Recognition/model.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=dev

In [13]:
!gdown 1hMVKCa84LLLj3_R6DcVtJRmV95YwTxFB

Downloading...
From (original): https://drive.google.com/uc?id=1hMVKCa84LLLj3_R6DcVtJRmV95YwTxFB
From (redirected): https://drive.google.com/uc?id=1hMVKCa84LLLj3_R6DcVtJRmV95YwTxFB&confirm=t&uuid=d03f851a-6a73-4e12-93ed-52e098b7acb1
To: /content/Handwritten-Mathematical-Expression-Recognition/checkpoint_11_8_512_30.pth
100% 327M/327M [00:01<00:00, 203MB/s]


In [14]:
num_encoder_layers = 11
nhead = 8
d_model = 256
dim_feedforward = 512
vocab_size = 257

model = OnHWRTransformer(vocab_size=vocab_size, clip_model=clip_model, d_model=d_model, nhead=nhead,
                         num_encoder_layers=num_encoder_layers,
                         dim_feedforward=dim_feedforward, dropout=0.1).to(device)

checkpoint = torch.load(f'checkpoint_{num_encoder_layers}_{nhead}_{dim_feedforward}_{30}.pth',
                        map_location=device)

model.load_state_dict(checkpoint['model'])

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
<ipython-input-14-e9667119a321>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you st

<All keys matched successfully>

In [15]:
valid_cer = evaluate(model, valid_loader, device)
print(valid_cer)

100%|██████████| 7644/7644 [03:47<00:00, 33.53it/s]

0.10472426033622718
